# install underthesea, gensim

In [2]:
# pip install underthesea

In [ ]:
#pip install gensim

Note: you may need to restart the kernel to use updated packages.


# Gọi 2 bộ dữ liệu

In [4]:
import pandas as pd

## UIT_VSMEC
Chỉ sử dụng train và valid

### Đọc dữ liệu từ folder data

In [5]:
uit_df = pd.read_csv("..\\data\\UIT_VSMEC\\vsmec_merged_train_valid.csv", encoding="utf-8-sig")

In [6]:
print(uit_df.shape)
uit_df['Emotion'].value_counts()

(6234, 2)


Emotion
Enjoyment    1772
Disgust      1206
Other        1162
Sadness      1033
Anger         440
Fear          349
Surprise      272
Name: count, dtype: int64

### Thống kê cơ bản

In [7]:
uit_df.describe()

,Sentence,Emotion
count,6234,6234
unique,6229,7
top,đã năm 2000 . người ta cho mượn 200 tr rồi .. ...,Enjoyment
freq,2,1772


In [8]:
print("Số lượng null trong toàn bộ dữ liệu UIT_VSMEC:\n", uit_df.isnull().sum())

Số lượng null trong toàn bộ dữ liệu UIT_VSMEC:
 Sentence    0
Emotion     0
dtype: int64


In [9]:
# Xem chi tiết 5 câu trong uit_df
for i in range(23, 28):
    print(f"Câu {i}: {uit_df.loc[i, 'Sentence']}")
    print(f"Nhãn: {uit_df.loc[i, 'Emotion']}")
    print()

Câu 23: yêu chết đi được .
Nhãn: Enjoyment

Câu 24: mày định lướt qua rồi nhưng khi đọc bình luận của các bạn và xem kĩ lại thì mày đã rớt nước mắt 😭😭😭.... người cha thật vĩ đại
Nhãn: Sadness

Câu 25: xin đi chơi với bạn ngày tổng kết không cho cứ bắt học miết :)) khá mệt
Nhãn: Sadness

Câu 26: cá nhân mình nghĩ nó dừng hay đỗ đúng sai thì đã có ca pạt còn nó đỗ trước cửa nhà mình mà oto hay xe không ra vào được thì lúc đó mới ý kiến . công nhận bọn oto đỗ trước nhà nhìn chỉ muốn đốt
Nhãn: Anger

Câu 27: từ đầu năm giờ chắc chụp 10 lần rồi huhu sợ quá 😭😭😭
Nhãn: Fear



\- Trong dữ liệu bình luận văn bản, sẽ có cả các câu rất ngắn và các câu rất dài

\- Những câu văn, mặc dù cái icon :)) , giống 'cười' nhưng lại chưa chắc câu đấy có nhãn Enjoyment hay không, ví dụ như câu 25.

\- :)) với :) là giống nhau, xử lý các ký tự lặp lại, tránh việc quá nhiều :)) hoặc :)))) dẫn đến tăng kích thước bộ từ điển :<

\- khi đọc bộ dữ liệu của UIT_VSMEC, phát hiện có các câu có dấu thanh bị tách rời, ví dụ: â &rarr; a^

In [10]:
# Ví dụ 1 dòng có dấu thanh bị tách rời
uit_df['Sentence'][15]

'tui thi ́ ch va ̉ i lă ́ mày ma ̀ ăn nhỉ ̀ u no ́ người lă ́ mày mọi người anh ̣ , bi ̣ lơ ̉ miê ̣ người'

In [11]:
# Tìm các index của dòng bị lỗi
import re

def find_rows_with_tone_mark_errors(df, column_name='Sentence'):
    tone_marks = r'[\u0300-\u036F]'
    
    error_indices = []
    for idx, text in df[column_name].items():
        if re.search(tone_marks, str(text)):
            error_indices.append(idx)
    
    return error_indices

In [12]:
print("Số lượng dòng có dấu thanh bị tách rời trong bộ dữ liệu:", len(find_rows_with_tone_mark_errors(uit_df)))

print("\nCác dòng có dấu thanh bị tách rời trong bộ dữ liệu:")
print(find_rows_with_tone_mark_errors(uit_df))

Số lượng dòng có dấu thanh bị tách rời trong bộ dữ liệu: 43

Các dòng có dấu thanh bị tách rời trong bộ dữ liệu:
[15, 172, 318, 350, 471, 489, 512, 557, 831, 832, 988, 1062, 1099, 1588, 1937, 1968, 2073, 2178, 2433, 2464, 2682, 2929, 2942, 3170, 3224, 3579, 3713, 4239, 4436, 4517, 4842, 4847, 4985, 5207, 5290, 5294, 5539, 5567, 5611, 5847, 5926, 5974, 6109]


### Kiểm tra trùng lặp

In [13]:
print("Số lượng dòng trùng lặp trong bộ dữ liệu:", len(uit_df[uit_df.duplicated(keep=False)]))

# In thử giá trị trùng lặp
uit_df[uit_df.duplicated(keep=False)].head(10)

Số lượng dòng trùng lặp trong bộ dữ liệu: 8


,Sentence,Emotion
848,đã năm 2000 . người ta cho mượn 200 tr rồi .. ...,Other
1030,bài này hay quá,Surprise
1128,per bây giờ thì không nói nhưng mà cảm giác ph...,Disgust
1275,per bây giờ thì không nói nhưng mà cảm giác ph...,Disgust
1936,nguy hiểm lắm,Fear
4386,nguy hiểm lắm,Fear
5092,đã năm 2000 . người ta cho mượn 200 tr rồi .. ...,Other
6140,bài này hay quá,Surprise


In [14]:
print("Số lượng dòng trùng lặp trong cột Sentence:", len(uit_df[uit_df['Sentence'].duplicated(keep=False)]))

# In thử giá trị trùng lặp
uit_df[uit_df['Sentence'].duplicated(keep=False)].head(10)

Số lượng dòng trùng lặp trong cột Sentence: 10


,Sentence,Emotion
848,đã năm 2000 . người ta cho mượn 200 tr rồi .. ...,Other
1030,bài này hay quá,Surprise
1128,per bây giờ thì không nói nhưng mà cảm giác ph...,Disgust
1275,per bây giờ thì không nói nhưng mà cảm giác ph...,Disgust
1936,nguy hiểm lắm,Fear
4116,kinh vãi,Disgust
4386,nguy hiểm lắm,Fear
4987,kinh vãi,Fear
5092,đã năm 2000 . người ta cho mượn 200 tr rồi .. ...,Other
6140,bài này hay quá,Surprise


## HUS_FB

### Đọc dữ liệu từ folder data

In [15]:
hus_fb_df = pd.read_csv("..\\data\\hus_fb.csv", encoding="utf-8-sig")

In [16]:
print(hus_fb_df.shape)
hus_fb_df['Emotion'].value_counts()

(4616, 2)


Emotion
Enjoyment    1812
Other         865
Disgust       619
Sadness       463
Anger         429
Surprise      304
Fear          124
Name: count, dtype: int64

### Thống kê cơ bản

In [17]:
hus_fb_df.describe()

,Sentence,Emotion
count,4616,4616
unique,4539,7
top,nam mô a di đà phật,Enjoyment
freq,11,1812


In [18]:
print("Số lượng null trong toàn bộ dữ liệu HUS_FB:\n", hus_fb_df.isnull().sum())

Số lượng null trong toàn bộ dữ liệu HUS_FB:
 Sentence    0
Emotion     0
dtype: int64


In [19]:
# Xem chi tiết 5 câu trong hus_bf
for i in range(711, 716):
    print(f"Câu {i}: {hus_fb_df.loc[i, 'Sentence']}")
    print(f"Nhãn: {hus_fb_df.loc[i, 'Emotion']}")
    print()

Câu 711: hình như bà vk nào cũng giống em hay sao ý 😆
Nhãn: Enjoyment

Câu 712: móc lỗ mũi 🤣🤣
Nhãn: Enjoyment

Câu 713: sự trả thù ngọt ngào 😆
Nhãn: Enjoyment

Câu 714: may quá em không có 🫣
Nhãn: Enjoyment

Câu 715: ê sao giọng giống mày vậy
Nhãn: Surprise



In [20]:
print("Số lượng dòng có dấu thanh bị tách rời trong bộ dữ liệu:", len(find_rows_with_tone_mark_errors(hus_fb_df)))

print("\nCác dòng có dấu thanh bị tách rời trong bộ dữ liệu:")
print(find_rows_with_tone_mark_errors(hus_fb_df))

Số lượng dòng có dấu thanh bị tách rời trong bộ dữ liệu: 1

Các dòng có dấu thanh bị tách rời trong bộ dữ liệu:
[1514]


In [21]:
hus_fb_df.loc[1514, 'Sentence']  # Ví dụ một dòng có dấu thanh bị tách rời

'riêng cái câu: tôi không muốn phán xét bla bla bla cơ bản là đã phán xét rồi :))) còn ráng chia ra là phán xét bề ngoài phán xét nhân phẩm hay là ở các nước mà bạn cho là đỉnh cao hoặc bất cứ đâu như: mỹ anh phápvv bạn có chắc không gặp cảnh này hoặc họ chỉ mặc bikini'

### Kiểm tra trùng lặp

In [22]:
print("Số lượng dòng trùng lặp trong bộ dữ liệu:", len(hus_fb_df[hus_fb_df.duplicated(keep=False)]))

# In thử giá trị trùng lặp
hus_fb_df[hus_fb_df.duplicated(keep=False)].head(10)

Số lượng dòng trùng lặp trong bộ dữ liệu: 110


,Sentence,Emotion
63,vãi lồn luôn,Surprise
81,ê nha,Surprise
98,1 rắn độc 🐍,Disgust
100,1 rắn độc 🐍,Disgust
270,chia buồn cùng gia đình,Sadness
274,xin chia buồn cùng gia đình,Sadness
275,a di đà phật,Sadness
389,thua cả mèo là có thật,Sadness
403,thua cả mèo là có thật,Sadness
447,cưng,Enjoyment


In [23]:
print("Số lượng dòng trùng lặp trong cột Sentence:", len(hus_fb_df[hus_fb_df['Sentence'].duplicated(keep=False)]))

# In thử giá trị trùng lặp
hus_fb_df[hus_fb_df['Sentence'].duplicated(keep=False)].head(10)

Số lượng dòng trùng lặp trong cột Sentence: 117


,Sentence,Emotion
63,vãi lồn luôn,Surprise
81,ê nha,Surprise
98,1 rắn độc 🐍,Disgust
100,1 rắn độc 🐍,Disgust
270,chia buồn cùng gia đình,Sadness
274,xin chia buồn cùng gia đình,Sadness
275,a di đà phật,Sadness
389,thua cả mèo là có thật,Sadness
403,thua cả mèo là có thật,Sadness
447,cưng,Enjoyment


# Các hàm xử lý dữ liệu

In [24]:
import re
from underthesea import text_normalize

## 1. Loại bỏ null, rỗng khỏi dữ liệu

In [25]:
# Loại bỏ null khỏi dữ liệu
def remove_null(df):
    df = df.dropna().reset_index(drop=True)
    return df

## 2. Chuẩn hóa về chữ thường

In [26]:
def convert_to_lowercase(text):
    return text.lower()

## 3. Chuyển ký tự lặp lại liên tiếp thành 1 ký tự đơn

In [27]:
def remove_repeated_characters(text):
    """
    Chuyển ký tự lặp lại liên tiếp thành 1 ký tự đơn
    Ví dụ:
    - "đẹpppp" -> "đẹp"
    - "woooow" -> "wow"
    - "yayyy" -> "yay"
    - ":))))" -> ":)"
    - ":((" -> ":("
    """
    
    # Chuyển ký tự lặp lại liên tiếp, chỉ giữ lại 1 ký tự
    # Pattern: ([character])\1+ nghĩa là ký tự nào xuất hiện 2 lần trở lên liên tiếp
    # Áp dụng cho: chữ cái (a-z, A-Z), chữ tiếng Việt có dấu, số, và các ký tự đặc biệt
    
    # Xử lý tất cả các ký tự (bao gồm cả ký tự đặc biệt như :, ), (, v.v.)
    cleaned_text = re.sub(
        r'(.)\1+',  # Bắt bất kỳ ký tự nào (.) xuất hiện 2 lần trở lên liên tiếp
        r'\1',      # Thay thế bằng chỉ 1 ký tự đó
        text
    )
    
    return cleaned_text    

## 4. Thay thế các emoji và emoticon (phổ biến) thành các từ

In [28]:
def replace_emoji_to_vietnamese(text):
    # Tokenize emoticon
    emoticon_map = {
        ':)': '<cười>', '(:': '<cười>', ':-)': '<cười>', '(-:': '<cười>', '=)': '<cười>', '(=': '<cười>',
        ':(': '<buồn>', '):': '<buồn>', ':-(': '<buồn>', ')-:': '<buồn>', '=(': '<buồn>', ')=': '<buồn>',
        ':D': '<cười_lớn>', 'xD': '<cười_lớn>', 'XD': '<cười_lớn>', '=D': '<cười_lớn>', ':d': '<cười_lớn>', '=d': '<cười_lớn>',
        ':P': '<đùa>', ':p': '<đùa>', ':-P': '<đùa>', ':-p': '<đùa>',
        ';)': '<nháy_mắt>', ';-)': '<nháy_mắt>', ';D': '<nháy_mắt_cười>',
        '<3': '<tim>',
        ':/': '<hoài_nghi>', ':\\': '<hoài_nghi>', ':-/': '<hoài_nghi>', ':-\\': '<hoài_nghi>',
        ':o': '<ngạc_nhiên>', ':O': '<ngạc_nhiên>', ':-o': '<ngạc_nhiên>', ':-O': '<ngạc_nhiên>',
        '>:(': '<tức_giận>', '>:-(': '<tức_giận>', 'D:<': '<tức_giận>',
        'T_T': '<khóc>', 'T.T': '<khóc>', 'ToT': '<khóc>', ';_;': '<khóc>', 'Q_Q': '<khóc>', 'T^T': '<khóc>',
        '^_^': '<vui>', '^-^': '<vui>', '^.^': '<vui>', '^o^': '<vui>',
        '-_-': '<mệt_mỏi>', '-.-': '<chán>', '=_=': '<ngái_ngủ>',
        'o_o': '<sốc>', 'O_O': '<sốc>', 'o.o': '<sốc>', 'O.O': '<sốc>',
        '>_<': '<khó_chịu>', '>.<': '<khó_chịu>', 'x_x': '<chết>', 'X_X': '<chết>',
        ':3': '<dễ_thương>', ':v': '<đùa>', '^^': '<vui>', '>.>': '<nghi_ngờ>', '<.<': '<nghi_ngờ>',
    }
    
    # Thay thế emoticon thông thường còn lại (theo độ dài giảm dần)
    sorted_emoticons = sorted(emoticon_map.keys(), key=len, reverse=True)
    for emoticon in sorted_emoticons:
        if emoticon in text:
            token = emoticon_map[emoticon]
            # Sử dụng regex để thay thế chính xác emoticon (tránh trùng lặp với các ký tự khác)
            text = re.sub(re.escape(emoticon), f" {token} ", text)
    
    # Emoji map
    emoji_map = {
        '😀': '<cười>', '😃': '<cười_rạng_rỡ>', '😄': '<cười_tươi>', '😁': '<cười_nhe_răng>',
        '😆': '<cười_toe_toét>', '🥲': '<nụ_cười_nước_mắt>', '🤣': '<cười_lăn_lộn_trên_sàn>', '😂': '<cười_chảy_nước_mắt>',
        '😅': '<cười_ngượng_ngùng>', '🥹': '<mặt_nước_mắt_rưng_rưng>', '☺️': '<mỉm_cười>', '😊': '<cười_mỉm>',
        '🙂': '<mỉm_cười>', '🙃': '<mỉm_cười_ngược>', '😘': '<hôn>', '🥰': '<yêu_thương>',
        '😍': '<say_mê>', '😌': '<hài_lòng>', '😉': '<nháy_mắt>', '😗': '<hôn_nhẹ>', '😙': '<hôn_dễ_thương>',
        '😚': '<hôn_dịu_dàng>', '😋': '<thích_thú>', '😛': '<trêu_chọc>', '🧐': '<suy_xét>', '🤨': '<nghi_ngờ>',
        '🤪': '<điên_rồ>', '😜': '<đùa_nghịch>', '😝': '<nhăn_nhở>', '😎': '<tự_tin>', '🥳': '<ăn_mừng>',
        '😟': '<lo_lắng>', '😔': '<trầm_ngâm>', '😞': '<thất_vọng>', '😒': '<chán>', '😏': '<cười_mỉa>',
        '😕': '<bối_rối>', '🙁': '<buồn>', '☹️': '<rầu_rĩ>', '😣': '<ráng_sức>', '😖': '<bối_rối_khổ_sở>',
        '😭': '<khóc_nức_nở>', '😢': '<buồn_bã>', '🥺': '<đáng_yêu>', '😩': '<thở_dài>', '😫': '<kiệt_sức>',
        '😤': '<hậm_hực>', '😠': '<giận>', '😡': '<phẫn_nộ>', '🤬': '<chửi_thề>', '🤯': '<kinh_ngạc>', '🤩': '<ngưỡng_mộ>',
        '😳': '<ngại_ngùng>', '🥵': '<nóng_nực>', '🥶': '<lạnh_cóng>', '😱': '<hoảng_sợ>', '🤗': '<thương>',
        '😓': '<ái_ngại>', '😥': '<buồn_nhưng_nhẹ_nhõm>', '😰': '<lo_sợ>', '😨': '<sợ_hãi>', '🤔': '<suy_nghĩ>',
        '🫣': '<rụt_rè>', '🤭': '<che_miệng>', '🫢': '<ngạc_nhiên>', '🫡': '<chào_quân_sự>', '🤫': '<suỵt>',
        '🫠': '<tan_chảy>', '🤥': '<nói_dối>', '😶': '<không_nói>', '🫥': '<ẩn_mình>', '😐': '<bình_thản>',
        '🫤': '<bối_rối>', '😑': '<vô_cảm>', '😬': '<khó_xử>', '🙄': '<mỉa_mai>', '😲': '<ngạc_nhiên>',
        '😮': '<há_mồm_ngạc_nhiên>', '😧': '<bàng_hoàng>', '😦': '<bối_rối>', '😯': '<ngạc_nhiên_nhẹ>', '🥱': '<ngáp>',
        '😴': '<ngủ>', '🤤': '<thèm_thuồng>', '😪': '<buồn_ngủ>', '😮‍💨': '<thở_dài>', '🤢': '<buồn_nôn>',
        '🥴': '<chóng_mặt>', '🤐': '<ngậm_miệng>', '😵‍💫': '<hoa_mắt>', '😵': '<chóng_mặt>', '🤮': '<nôn_mửa>',
        '🤡': '<hề_hài>', '🐧': '<đùa>', '💔': '<tim_tan_nát>', '❤️‍🩹': '<tim_được_chữa_lành>', '🫂': '<ôm>',
        '👍': '<tán_thành>', '👌': '<đồng_ý>', '🙏': '<cảm_ơn>', '👏': '<vỗ_tay>',
        '💖': '<tim>', '💕': '<tim>', '💞': '<tim>', '💓': '<tim>', '💓': '<tim>', '💗': '<tim>',
        '💯': '<một_trăm_điểm>', '🔥': '<tuyệt_vời>', 
        '❤️': '<tim>', '🩷': '<tim>', '🧡': '<tim>', '💛': '<tim>', '💚': '<tim>', '💙': '<tim>', 
        '🩵': '<tim>', '💜': '<tim>', '🤎': '<tim>', '🖤': '<tim>', '🩶': '<tim>', '🤍': '<tim>',
        '🎉': '<ăn_mừng>', '😇': '<thiên_thần>', '😈': '<ác_quỷ>',
        '🫶': '<tim_bằng_tay>', '🙌': '<vui_mừng>', '🫰': '<tim_bằng_tay>',
        '✌️': '<chiến_thắng>', '🤟': '<tình_yêu_rock_and_roll>', '👋': '<vẫy_tay_chào>'
    }

    # Sắp xếp emoji theo độ dài giảm dần để đảm bảo các emoji nhiều ký tự được thay thế trước
    sorted_emojis = sorted(emoji_map.keys(), key=len, reverse=True)
    # Kiểu có mấy cái emoji ❤️\u200d🩹 = ❤️‍🩹

    for emoji in sorted_emojis:
        if emoji in text:
            token = emoji_map[emoji]
            # Thay thế tất cả các lần xuất hiện của emoji bằng token và thêm khoảng trắng xung quanh
            text = text.replace(emoji, f"{token}")

    # Loại bỏ các token emoji lặp lại liên tiếp (ví dụ: <tim> <tim> -> <tim>)
    for token in set(emoji_map.values()):
        text = text.replace(f"{token} {token}", token)
    
    return text

## 5. Chuẩn hóa từ viết tắt / từ lóng (phổ biến)

In [29]:
def normalize_short_forms(text):
    short_forms_map = {
        'đm': '<địt_mẹ>', 'dm': '<địt_mẹ>', 'vcl': '<vãi_cả_lồn>', 'clgt': '<cái_lồn_gì_thế>',
        'lol': '<cười_lớn>', 'xl': '<xin_lỗi>', 'ko': '<không>', 'k': '<không>', 'r': '<rồi>',
        'bn': '<bạn>', 'vs': '<với>', 'bik': '<biết>', 'nx': '<nữa>', 'qá': '<quá>', 'cj': '<chị>',
        'ad': '<quản_trị_viên>', 'acc': '<tài_khoản>', 'cmt': '<bình_luận>', 'clm': '<cái_lồn_má>',
        'rep': '<trả_lời>', 'ib': '<nhắn_tin_riêng>', 'cmnl': '<con_mẹ_nó_luôn>', 'vc': '<vãi_chưởng>',
        'cc': '<con_cặc>', 'ditme': '<địt_mẹ>', 'vl': '<vãi_lồn>', 'm': '<mày>', 'mk': '<mình>', 'vloz': '<vãi_lồn>',
        'mn': '<mọi_người>', 'nc': '<nói_chuyện>', 'ng': '<người>', 'nh': '<nhắn>', 'n9': '<nam_chính>', 'nu9': '<nữ_chính>',
        'thik': '<thích>', 'uk': '<ừ>', 'j': '<gì>', 'zậy': '<vậy>',
        'nhe': '<nhé>', 'cug': '<cũng>', 'dc': '<được>', 'đc': '<được>', 'in4': '<thông_tin>',
        'thg': '<thằng>', 'ngta': '<người_ta>', 'khum': '<không>', 'khôm': '<không>',
        'lun': '<luôn>', 'thui': '<thôi>', 'thoy': '<thôi>', 'iu': '<yêu>',
        'gato': '<ghen_ăn_tức_ở>', 'đx': '<được>', 'bthg': '<bình_thường>',
        'bt': '<biết>', 'hk': '<không>', 'qtrg': '<quan_trọng>', 'lm': '<lắm>',
        'p/s': '<tái_bút>', 'ibox': '<nhắn_tin_riêng>', 'bjo': '<bao_giờ>', 'thks': '<cảm_ơn>',
        'tk': '<thằng>', 'đr': '<đúng_rồi>', 'cmm': '<con_mẹ_mày>', 'đmm': '<địt_mẹ_mày>',
        'vcđ': '<vãi_cả_đái>', 'vch': '<vãi_cả_chưởng>', 'ck': '<chồng>', 'vk': '<vợ>', 'ny': '<người_yêu>', 'nyc': '<người_yêu_cũ>',
        'nt': '<nhắn_tin>', 'loz': '<lồn>', 'dkm': '<địt_con_mẹ>',
        'dkmm': '<địt_con_mẹ_mày>', 'moasss': '<má>'
    }

    # Sắp xếp các từ viết tắt theo độ dài giảm dần để tránh việc thay thế một phần của từ viết tắt khác
    sorted_short_forms = sorted(short_forms_map.keys(), key=len, reverse=True)
    for short_form in sorted_short_forms:
        if short_form in text:
            full_form = short_forms_map[short_form]
            # Sử dụng regex để thay thế chính xác từ viết tắt (tránh trùng lặp với các từ khác)
            text = re.sub(r'\b' + re.escape(short_form) + r'\b', full_form, text)

    return text

## 6. Xóa các dòng có dấu thanh bị tách rời

In [30]:
def remove_rows_with_separated_tone_marks(df, column_name='Sentence'):
    """
    Xóa các dòng có dấu thanh bị tách rời khỏi dữ liệu
    """
    
    # Dấu thanh Unicode combining (U+0300 - U+036F)
    tone_marks = r'[\u0300-\u036F]'  # Bao gồm: ́ ̀ ̉ ̃ ̣
    
    # Tạo mask để lọc các dòng KHÔNG chứa dấu thanh tách rời
    # Áp dụng hàm kiểm tra một chuỗi có dấu thanh tách rời không
    mask = df[column_name].apply(lambda text: not bool(re.search(tone_marks, str(text))))
    
    # Lọc DataFrame
    df_cleaned = df[mask].copy().reset_index(drop=True)
    
    return df_cleaned

## 7. Xóa các ký tự đặc biệt 
*(trừ các dấu _ . : ; , ! ?)*

In [31]:
# Hàm để loại bỏ các ký tự KHÔNG PHẢI chữ cái tiếng Việt và Latin, số, '_', '.', ',', ';', '!', '?', ':'
def remove_non_alphanumeric(text):
    if pd.isnull(text) or text == "":
        return text
    # Giữ lại những ký tự mong muốn 
    cleaned_text = re.sub(r'[^0-9_.:;,!?a-zđáàảạãăắằẳặẵâấầẩậẫéèẻẹẽêềếểệễíìỉịĩóòỏọõôốồổộỗơờớởợỡúùủụũưứừửựữýỳỷỵỹ]', ' ', text)

    return cleaned_text.strip()

## 8. Chuẩn hóa chính tả
Sử dụng hàm `text_normalize` trong `underthesea`
- Chuẩn hóa Unicode
- Sửa vị trí của dấu câu: 'hoà' -> 'hòa'
- Loại bỏ khoảng trắng thừa

In [32]:
def normalize_text(text):
    # Chuẩn hóa Unicode và sửa vị trí dấu câu, loại bỏ khoảng trắng thừa
    text = text_normalize(text)
    
    return text

## 9. Kiểm tra và loại bỏ trùng lặp

Loại bỏ trùng lặp, xét trên cột Sentence, để tránh nhầm lẫn khi máy học

In [33]:
def remove_duplicates_sentence(df):
    df = df.drop_duplicates(subset='Sentence').reset_index(drop=True)
    return df

## 10. Tokenize

In [34]:
# Tokenize: từ các bình luận, chuyển thành các token
from underthesea import word_tokenize

def tokenize_comments(df):
    df['Tokenized_Sentence'] = df['Sentence'].apply(word_tokenize)
    return df

## 11. Loại bỏ các dấu _ . : ; ,

In [35]:
# Loại bỏ các dấu _ . : ; , sau khi tokenize
def remove_specific_punctuations_after_tokenization(tokens):
    punctuations_to_remove = {'_', '.', ':', ';', ','}
    cleaned_tokens = [token for token in tokens if token not in punctuations_to_remove]
    return cleaned_tokens

## 12. Loại bỏ từ dừng

In [36]:
vietnamese_stopwords = set([
    'và', 'là', 'có', 'của', 'cho', 'với', 'rằng', 'thì', 'mà', 'lại',
    'này', 'đã', 'đang', 'sẽ', 'được', 'bị', 'ở', 'trong', 'ra', 
    'khi', 'một', 'các', 'những', 'nên', 'vì', 'như', 'thôi', 'nữa',
    'đến', 'đi', 'rất', 'quá', 'thật', 'đó', 'này', 'kia', 'vậy', 'rồi'
])

def delete_stopwords(tokens): 
    return [word for word in tokens if word not in vietnamese_stopwords]

## 13. Loại bỏ các dòng rỗng ""

In [37]:
def remove_nan(df):
    df = df[df['Sentence'].str.strip() != ""].reset_index(drop=True)
    df = df[df['Tokenized_Sentence'].map(lambda tokens: len(tokens) > 0)].reset_index(drop=True)
    return df

# Pipeline xử lý dữ liệu

In [38]:
def pipeline_preprocessing(df):
    # 1. Loại bỏ null khỏi dữ liệu
    df = remove_null(df)

    # 2. Chuyển tất cả văn bản thành chữ thường
    df['Sentence'] = df['Sentence'].apply(convert_to_lowercase)
    
    # 3. Chuyển ký tự lặp lại liên tiếp thành 1 ký tự đơn
    df['Sentence'] = df['Sentence'].apply(remove_repeated_characters)
    
    # 4. Thay thế các emoji và emoticon thành các từ
    df['Sentence'] = df['Sentence'].apply(replace_emoji_to_vietnamese)
    
    # 5. Chuẩn hóa từ viết tắt, từ lóng
    df['Sentence'] = df['Sentence'].apply(normalize_short_forms)

    # 6. Xóa các dòng có dấu thanh bị tách rời khỏi dữ liệu
    df = remove_rows_with_separated_tone_marks(df)

    # 7. Xóa các ký tự đặc biệt (trừ các dấu _.;,!?)
    df['Sentence'] = df['Sentence'].apply(remove_non_alphanumeric)

    # 8. Chuẩn hóa chính tả
    df['Sentence'] = df['Sentence'].apply(normalize_text)

    # 9. Loại bỏ trùng lặp, xét trên cột Sentence
    df = remove_duplicates_sentence(df)

    # 10. Tokenize: từ các bình luận, chuyển thành các token
    df = tokenize_comments(df)

    # 11. Loại bỏ các dấu _ . : ; , sau khi tokenize
    df['Tokenized_Sentence'] = df['Tokenized_Sentence'].apply(remove_specific_punctuations_after_tokenization)

    # 12. Loại bỏ từ dừng 
    df['Tokenized_Sentence'] = df['Tokenized_Sentence'].apply(delete_stopwords)
    
    # 13. Loại bỏ các dòng rỗng ""
    df = remove_nan(df)


    return df

# Chạy pipeline

## UIT_VSMEC

In [38]:
processed_uit = pipeline_preprocessing(uit_df)

In [39]:
processed_uit.describe()

,Sentence,Emotion,Tokenized_Sentence
count,6183,6183,6183
unique,6183,7,6166
top,dume lũ mất dạy . bố mẹ cho mày ăn học để dơ n...,Enjoyment,[nguy hiểm]
freq,1,1754,5


In [40]:
print("Số lượng dòng bị loại bỏ sau khi xử lý:", uit_df.shape[0] - processed_uit.shape[0])
print("Kích thước bộ dữ liệu sau khi xử lý:", processed_uit.shape)

Số lượng dòng bị loại bỏ sau khi xử lý: 51
Kích thước bộ dữ liệu sau khi xử lý: (6183, 3)


In [41]:
print("Số lượng null trong bộ dữ liệu sau khi xử lý:")
processed_uit.isnull().sum()

Số lượng null trong bộ dữ liệu sau khi xử lý:


Sentence              0
Emotion               0
Tokenized_Sentence    0
dtype: int64

In [42]:
processed_uit['Emotion'].value_counts()

Emotion
Enjoyment    1754
Disgust      1192
Other        1158
Sadness      1027
Anger         436
Fear          346
Surprise      270
Name: count, dtype: int64

In [43]:
processed_uit.head(5)

,Sentence,Emotion,Tokenized_Sentence
0,cho mình xin bài nhạc tên là gì với ạ,Other,"[mình, xin, bài, nhạc, tên, gì, ạ]"
1,cho đáng đời con quỷ . về nhà lôi con nhà mày ...,Disgust,"[đáng, đời, con, quỷ, về, nhà, lôi, con, nhà, ..."
2,lo học đi . yêu đương cười_lớn gì hay lại thíc...,Disgust,"[lo, học, yêu, đương cười_lớn, gì, hay, thích,..."
3,ước gì sau này về già vẫn có thể như cụ này cười,Enjoyment,"[ước, gì, sau này, về, già, vẫn, có thể, cụ, c..."
4,mỗi lần có video của con là cứ coi đi coi lại ...,Enjoyment,"[mỗi, lần, video, con, cứ, coi, coi, hoài, cưn..."


In [44]:
path = "..\\data\\preprocessed_data"

processed_uit.to_csv(f"{path}\\preprocessed_uit.csv",  index=False, encoding="utf-8-sig")

## HUS_FB

In [53]:
processed_hus_fb = pipeline_preprocessing(hus_fb_df)

In [54]:
processed_hus_fb.describe()

,Sentence,Emotion,Tokenized_Sentence
count,4521,4521,4521
unique,4521,7,4497
top,đảm bảo an sinh xã hội công bằng cho tất cả mọ...,Enjoyment,[dễ thương]
freq,1,1756,6


In [55]:
print("Số lượng dòng bị loại bỏ sau khi xử lý:", hus_fb_df.shape[0] - processed_hus_fb.shape[0])
print("Kích thước bộ dữ liệu sau khi xử lý:", processed_hus_fb.shape)

Số lượng dòng bị loại bỏ sau khi xử lý: 95
Kích thước bộ dữ liệu sau khi xử lý: (4521, 3)


In [56]:
print("Số lượng null trong bộ dữ liệu sau khi xử lý:")
processed_hus_fb.isnull().sum()

Số lượng null trong bộ dữ liệu sau khi xử lý:


Sentence              0
Emotion               0
Tokenized_Sentence    0
dtype: int64

In [57]:
processed_hus_fb['Emotion'].value_counts()

Emotion
Enjoyment    1756
Other         860
Disgust       616
Sadness       445
Anger         428
Surprise      293
Fear          123
Name: count, dtype: int64

In [58]:
processed_hus_fb.head(5)

,Sentence,Emotion,Tokenized_Sentence
0,xin lỗi chứ loại này tôi đéo bao giờ chơi lần ...,Disgust,"[xin lỗi, chứ, loại, tôi, đéo, bao giờ, chơi, ..."
1,đã sợ bị đánh thì đừng có nói cười,Anger,"[sợ, đánh, đừng, nói, cười]"
2,đã sợ đừng có nói cười bản lĩnh cái thằng đàn ...,Anger,"[sợ, đừng, nói, cười, bản lĩnh, cái, thằng, đà..."
3,chắc cuộc đời mày chưa bao giờ nói xấu ai chứ ...,Anger,"[chắc, cuộc đời, mày, chưa, bao giờ, nói xấu, ..."
4,biết vậy nhưng cái mỏ nó giật lắm mày cười_lăn...,Disgust,"[biết, nhưng, cái, mỏ, nó, giật, lắm, mày, cườ..."


In [59]:
path = "..\\data\\preprocessed_data"

processed_hus_fb.to_csv(f"{path}\\preprocessed_hus_fb.csv",  index=False, encoding="utf-8-sig")

## TEST

In [52]:
test = pd.read_csv("..\\data\\UIT_VSMEC\\vsmec_test.csv", encoding="utf-8-sig")

In [53]:
processed_test = pipeline_preprocessing(test)

In [54]:
processed_test.describe()

,Sentence,Emotion,Tokenized_Sentence
count,688,688,688
unique,688,7,688
top,đôi mắt đằm thắm vãi,Enjoyment,"[đôi, mắt, đằm thắm, vãi]"
freq,1,192,1


In [55]:
print("Số lượng dòng bị loại bỏ sau khi xử lý:", test.shape[0] - processed_test.shape[0])
print("Kích thước bộ dữ liệu sau khi xử lý:", processed_test.shape)

Số lượng dòng bị loại bỏ sau khi xử lý: 5
Kích thước bộ dữ liệu sau khi xử lý: (688, 3)


In [56]:
print("Số lượng null trong bộ dữ liệu sau khi xử lý:")
processed_test.isnull().sum()

Số lượng null trong bộ dữ liệu sau khi xử lý:


Sentence              0
Emotion               0
Tokenized_Sentence    0
dtype: int64

In [57]:
processed_test['Emotion'].value_counts()

Emotion
Enjoyment    192
Other        129
Disgust      129
Sadness      115
Fear          46
Anger         40
Surprise      37
Name: count, dtype: int64

In [58]:
processed_test.head(5)

,Sentence,Emotion,Tokenized_Sentence
0,người ta có bạn bè nhìn vui thật,Sadness,"[người, ta, bạn bè, nhìn, vui]"
1,cho nghỉ viêc mói đúng sao goi là kỷ luật,Surprise,"[nghỉ, viêc, mói, đúng, sao, goi, kỷ luật]"
2,kinh vãi phẫn_nộ,Disgust,"[kinh vãi, phẫn_nộ]"
3,nhà thì không xa lắm nhưng chưa bao giờ đi vì ...,Fear,"[nhà, không, xa, lắm, nhưng, chưa, bao giờ, sợ..."
4,bố không thích nộp đấy mày thích ý kiến không,Anger,"[bố, không, thích, nộp, đấy, mày, thích, ý kiế..."


In [59]:
path = "..\\data\\preprocessed_data"

processed_test.to_csv(f"{path}\\preprocessed_test.csv",  index=False, encoding="utf-8-sig")

## MERGED

In [39]:
merged = pd.read_csv("..\\data\\merged_data\\merged_data.csv", encoding="utf-8-sig")

In [40]:
merged.shape

(8420, 3)

In [51]:
print("Số lượng dòng trùng lặp trong bộ dữ liệu:", len(merged[merged.duplicated(keep=False)]))

# In thử giá trị trùng lặp
merged[merged.duplicated(keep=False)].head(10)

Số lượng dòng trùng lặp trong bộ dữ liệu: 6


,Sentence,Emotion,Tokenized_Sentence
3154,sợ thế,Fear,"['sợ', 'thế']"
3489,đọc bình luận cười ẻ,Enjoyment,"['đọc', 'bình luận', 'cười', 'ẻ']"
3851,dễ thương,Enjoyment,"['dễ', 'thương']"
5052,sợ thế,Fear,"['sợ', 'thế']"
6255,đọc bình luận cười ẻ,Enjoyment,"['đọc', 'bình luận', 'cười', 'ẻ']"
7100,dễ thương,Enjoyment,"['dễ', 'thương']"


In [52]:
print("Số lượng dòng trùng lặp trong cột Sentence:", len(merged[merged['Sentence'].duplicated(keep=False)]))

# In thử giá trị trùng lặp
merged[merged['Sentence'].duplicated(keep=False)].head(10)

Số lượng dòng trùng lặp trong cột Sentence: 10


,Sentence,Emotion,Tokenized_Sentence
1283,hú hồn,Fear,"['hú', 'hồn']"
3154,sợ thế,Fear,"['sợ', 'thế']"
3489,đọc bình luận cười ẻ,Enjoyment,"['đọc', 'bình luận', 'cười', 'ẻ']"
3497,hú hồn,Surprise,"['hú', 'hồn']"
3851,dễ thương,Enjoyment,"['dễ', 'thương']"
3855,ghê vậy,Surprise,['ghê']
5052,sợ thế,Fear,"['sợ', 'thế']"
6255,đọc bình luận cười ẻ,Enjoyment,"['đọc', 'bình luận', 'cười', 'ẻ']"
7100,dễ thương,Enjoyment,"['dễ', 'thương']"
7654,ghê vậy,Disgust,['ghê']


In [50]:
merged.head()

,Sentence,Emotion,Tokenized_Sentence
0,không biết crush bây giờ có ở nhà không nữa kh...,Sadness,"['không', 'biết', 'crush', 'bây giờ', 'nhà', '..."
1,tại do bạn thoại kêu gọi chứ có phải mình chị ...,Disgust,"['tại', 'do', 'bạn thoại', 'kêu gọi', 'chứ', '..."
2,ai biểu đã vất vả rồi còn hám gái lo làm ăn đi...,Anger,"['ai', 'biểu', 'vất vả', 'còn', 'hám', 'gái', ..."
3,tao cũng phải về mở cho mẹ tao coi cười_chảy_n...,Enjoyment,"['tao', 'cũng', 'phải', 'về', 'mở', 'mẹ', 'tao..."
4,người anh em thiện lành đã rời việt nam . say ...,Other,"['người', 'anh em', 'thiện lành', 'rời', 'việt..."


In [42]:
processed_merged = pipeline_preprocessing(merged)

In [43]:
processed_merged.describe()

,Sentence,Emotion,Tokenized_Sentence
count,8415,8415,8415
unique,8415,7,8392
top,địt_mẹ hú hồn,Enjoyment,[nguy hiểm]
freq,1,2190,5


In [44]:
print("Số lượng dòng bị loại bỏ sau khi xử lý:", merged.shape[0] - processed_merged.shape[0])
print("Kích thước bộ dữ liệu sau khi xử lý:", processed_merged.shape)

Số lượng dòng bị loại bỏ sau khi xử lý: 5
Kích thước bộ dữ liệu sau khi xử lý: (8415, 3)


In [45]:
print("Số lượng null trong bộ dữ liệu sau khi xử lý:")
processed_merged.isnull().sum()

Số lượng null trong bộ dữ liệu sau khi xử lý:


Sentence              0
Emotion               0
Tokenized_Sentence    0
dtype: int64

In [46]:
processed_merged['Emotion'].value_counts()

Emotion
Enjoyment    2190
Other        1587
Disgust      1498
Sadness      1248
Anger         862
Surprise      562
Fear          468
Name: count, dtype: int64

In [47]:
processed_merged.head(5)

,Sentence,Emotion,Tokenized_Sentence
0,không biết crush bây giờ có ở nhà không nữa kh...,Sadness,"[không, biết, crush, bây giờ, nhà, không, khóc..."
1,tại do bạn thoại kêu gọi chứ có phải mình chị ...,Disgust,"[tại, do, bạn thoại, kêu gọi, chứ, phải, mình,..."
2,ai biểu đã vất vả rồi còn hám gái lo làm ăn đi...,Anger,"[ai, biểu, vất vả, còn, hám, gái, lo, làm ăn, ..."
3,tao cũng phải về mở cho mẹ tao coi cười_chảy_n...,Enjoyment,"[tao, cũng, phải, về, mở, mẹ, tao, coi, cười_c..."
4,người anh em thiện lành đã rời việt nam . say ...,Other,"[người, anh em, thiện lành, rời, việt nam, say..."


In [48]:
path = "..\\data\\preprocessed_data"

processed_merged.to_csv(f"{path}\\preprocessed_merged.csv",  index=False, encoding="utf-8-sig")